# Data Cleaning

## import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from imperio import BoxCoxTransformer, ZCATransformer
from imperio import FrequencyImputationTransformer, TargetImputationTransformer
from sklearn.impute import KNNImputer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")


## Import the dataset

In [4]:
df = pd.read_csv("data1.csv")

# converting to date type
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
df = df.drop('Volume Sold (Gallons)', axis=1)

In [6]:
# check for missing values
df.isnull().sum()

Invoice/Item Number         0
Date                        0
Store Number                0
Store Name                  0
Address                  2376
City                     2375
Zip Code                 2420
Store Location           2375
County Number           79178
County                  79178
Category                 8020
Category Name           16086
Vendor Number               3
Vendor Name                 1
Item Number                 0
Item Description            0
Pack                        0
Bottle Volume (ml)          0
State Bottle Cost          10
State Bottle Retail        10
Bottles Sold                0
Sale (Dollars)             10
Volume Sold (Liters)        0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12591077 entries, 0 to 12591076
Data columns (total 23 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Invoice/Item Number   object 
 1   Date                  object 
 2   Store Number          int64  
 3   Store Name            object 
 4   Address               object 
 5   City                  object 
 6   Zip Code              object 
 7   Store Location        object 
 8   County Number         float64
 9   County                object 
 10  Category              float64
 11  Category Name         object 
 12  Vendor Number         float64
 13  Vendor Name           object 
 14  Item Number           int64  
 15  Item Description      object 
 16  Pack                  int64  
 17  Bottle Volume (ml)    int64  
 18  State Bottle Cost     object 
 19  State Bottle Retail   object 
 20  Bottles Sold          int64  
 21  Sale (Dollars)        object 
 22  Volume Sold (Liters)  float64
dtypes: fl

In [8]:
# Getting rid of $ in each value of column and change the type to floats
df['State Bottle Cost'] = df['State Bottle Cost'].str.replace('$', '', regex=True).astype('float')
df['State Bottle Retail'] = df['State Bottle Retail'].str.replace('$', '', regex=True).astype('float')
df['Sale (Dollars)'] = df['Sale (Dollars)'].str.replace('$', '', regex=True).astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12591077 entries, 0 to 12591076
Data columns (total 23 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Invoice/Item Number   object 
 1   Date                  object 
 2   Store Number          int64  
 3   Store Name            object 
 4   Address               object 
 5   City                  object 
 6   Zip Code              object 
 7   Store Location        object 
 8   County Number         float64
 9   County                object 
 10  Category              float64
 11  Category Name         object 
 12  Vendor Number         float64
 13  Vendor Name           object 
 14  Item Number           int64  
 15  Item Description      object 
 16  Pack                  int64  
 17  Bottle Volume (ml)    int64  
 18  State Bottle Cost     float64
 19  State Bottle Retail   float64
 20  Bottles Sold          int64  
 21  Sale (Dollars)        float64
 22  Volume Sold (Liters)  float64
dtypes: fl

Now these columns are no longer object type, they are float.

In [9]:
df.loc[df['Invoice/Item Number'].str.contains('03560500002', case=False, na=False) == True]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters)
7447989,S03560500002,01/12/2012,3869,Bootleggin' Barzini's Fin,412 1ST AVE,CORALVILLE,52241,"412 1ST AVE\nCORALVILLE 52241\n(41.672672, -91...",52.0,Johnson,...,"Sazerac Co., Inc.",41991,Firefly Skinny Tea,12,750,10.00,15.00,2,30.00,1.50
11051894,INV-03560500002,03/02/2017,9901,Cedar Ridge,1441 Marak Rd NW,Swisher,52338.0,"1441 Marak Rd NW\nSwisher 52338\n(41.832107, -...",77.0,Polk,...,CEDAR RIDGE VINEYARDS LL,74772,Howdys Apple Pie,6,750,8.89,13.34,1,80.04,0.75


As I saw the number after S or INV- can coincide, so I cannot delete S or INV- from the column.

I think this column is meaningless and I can drop it.

In [10]:
df = df.drop('Invoice/Item Number', axis=1)

I can also drop the categorical columns that have the same meaning as the numerical columns(example: County number, County name).

In [11]:
df = df.drop(['County', 'Category Name', 'Vendor Name', 'Store Name','Address', 'Item Description', 'Store Location'], axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12591077 entries, 0 to 12591076
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Date                  object 
 1   Store Number          int64  
 2   City                  object 
 3   Zip Code              object 
 4   County Number         float64
 5   Category              float64
 6   Vendor Number         float64
 7   Item Number           int64  
 8   Pack                  int64  
 9   Bottle Volume (ml)    int64  
 10  State Bottle Cost     float64
 11  State Bottle Retail   float64
 12  Bottles Sold          int64  
 13  Sale (Dollars)        float64
 14  Volume Sold (Liters)  float64
dtypes: float64(7), int64(5), object(3)
memory usage: 1.4+ GB


Zip code shoul not be object type

In [13]:
df['Zip Code'] = df['Zip Code'].replace('712-2', 7122)   

cannot convert from object to int directly so I convert to float

In [14]:
df['Zip Code'] = df['Zip Code'].str.replace('.0', '', regex=True).astype(float)

In [15]:
df = df.dropna(subset = ['Zip Code'])

now convert to int with the same size

In [16]:
df['Zip Code'] = df['Zip Code'].astype('Int64')

In [17]:
df['Zip Code'].unique()

<IntegerArray>
[52632, 51632, 52627,   522, 52342,   525,     2, 52761,   511,   321,
 ...
 52347,   456, 51453, 51653,   237,   634,   169, 51573, 51463, 52346]
Length: 337, dtype: Int64

In [18]:
# I can also do this for the Store Number as the max is 9932 
# df['Store Number'].max()
df['Store Number'] = df['Store Number'].astype('Int16')
# np.nanmax(df['County Number'].unique()) = 99.0
df['County Number'] = df['County Number'].astype('Int64')
# df['County Number'] = df['County Number'].astype('Int16')
# np.nanmax(df['Category'].unique()) = 1901200.0
df['Category'] = df['Category'].astype('Int64')
# df['Category'] = df['Category'].astype('Int32')
# np.nanmax(df['Vendor Number'].unique()) = 987.0
df['Vendor Number'] = df['Vendor Number'].astype('Int64')
# df['Vendor Number'] = df['Vendor Number'].astype('Int16')
# np.nanmax(df['Item Number'].unique()) = 999275
df['Item Number'] = df['Item Number'].astype('Int32')
# np.nanmax(df['Pack'].unique()) = 600
df['Pack'] = df['Pack'].astype('Int16')
# np.nanmax(df['Bottles Sold'].unique()) = 15000
df['Bottles Sold'] = df['Bottles Sold'].astype('Int16')

In [19]:
# check for missing values
df.isnull().sum()

Date                        0
Store Number                0
City                        0
Zip Code                    0
County Number           57421
Category                 2048
Vendor Number               2
Item Number                 0
Pack                        0
Bottle Volume (ml)          0
State Bottle Cost          10
State Bottle Retail        10
Bottles Sold                0
Sale (Dollars)             10
Volume Sold (Liters)        0
dtype: int64

I saw that Bottle Volume is in ml, so I will convert it into liters

In [20]:
df['Bottle Volume (ml)'] = df['Bottle Volume (ml)']/1000 
df.rename(columns = {'Bottle Volume (ml)' : 'Bottle Volume'}, inplace = True)

There are records where the Sale, Cost and Retail are 0. So I need to get rid of this errors.

In [21]:
# Total numbers of ambiguous data.
df['Sale (Dollars)'].isin([0]).sum()

3491

In [22]:
# Dropping the ambigous data.
df = df[df['Sale (Dollars)'].isin([0]) == False]

I decided to drop the rows where the County number is nan as it will just mess with the model and for such a big dataset deleting nearly one percent is no problem.

In [23]:
df = df.dropna(subset = ['County Number'])

I wanted to do this, but after droping nan in County Number, there were no missing data in the City column.

In [24]:
!photo_1(https://drive.google.com/file/d/1ymkjefv4F5dcFHuGQWsnPxDKYi5kmy5T/view?usp=share_link)
!photo_2(https://drive.google.com/file/d/1jByBDI8whOGhOORj_fNuz5yQUAxqVetx/view?usp=share_link)

/bin/bash: -c: line 0: syntax error near unexpected token `https://drive.google.com/file/d/1ymkjefv4F5dcFHuGQWsnPxDKYi5kmy5T/view?usp=share_link'
/bin/bash: -c: line 0: `photo_1(https://drive.google.com/file/d/1ymkjefv4F5dcFHuGQWsnPxDKYi5kmy5T/view?usp=share_link)'
/bin/bash: -c: line 0: syntax error near unexpected token `https://drive.google.com/file/d/1jByBDI8whOGhOORj_fNuz5yQUAxqVetx/view?usp=share_link'
/bin/bash: -c: line 0: `photo_2(https://drive.google.com/file/d/1jByBDI8whOGhOORj_fNuz5yQUAxqVetx/view?usp=share_link)'


I'm dropping the date since it just helped for data exploration.

In [25]:
df = df.drop('Date', axis =1)  

i need to reindexate the dataset

In [26]:
df.reset_index(drop=True, inplace=True)

After so research, I decided to include a column Profit Margin(revenue minus expenses) as from business perspective it will a have a greater correlation with my target column.

In [27]:
df['Profit Margin'] = (df['State Bottle Retail'] - df['State Bottle Cost']) * df['Bottles Sold']

Now I need to drop the columns I used for the Profit Margin, in order not to have multicoliniarity.

In [28]:
df = df.drop(['State Bottle Retail', 'State Bottle Cost', 'Bottles Sold'], axis=1)

In [29]:
df

,Store Number,City,Zip Code,County Number,Category,Vendor Number,Item Number,Pack,Bottle Volume,Sale (Dollars),Volume Sold (Liters),Profit Margin
0,2191,KEOKUK,52632,56,<NA>,255,297,6,0.75,162.84,4.50,54.3
1,2205,CLARINDA,51632,73,<NA>,255,297,6,0.75,325.68,9.00,108.6
2,3549,FORT MADISON,52627,56,<NA>,130,249,20,0.15,19.20,0.30,6.4
3,2513,IOWA CITY,522,52,<NA>,65,237,3,1.75,160.02,5.25,53.37
4,3942,TOLEDO,52342,86,<NA>,130,249,20,0.15,19.20,0.30,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...
10022925,3704,Emmetsburg,536,74,1081200,434,68846,12,0.75,11.54,0.75,3.85
10022926,3704,Emmetsburg,536,74,1081200,330,80457,12,1.00,10.85,3.00,10.86
10022927,3704,Emmetsburg,536,74,1081200,260,68306,12,0.75,9.75,0.75,3.25
10022928,3704,Emmetsburg,536,74,1081100,65,67557,12,1.00,12.59,6.00,25.2


Solving the capitalization problem in City column

In [30]:
df['City'] = df['City'].str.upper()

I think that doing the dummies on the City column, can create columns with greater correlation with the target column. 

In [31]:
df['City'].value_counts()[:5]

DES MOINES      889837
CEDAR RAPIDS    664534
DAVENPORT       448940
WATERLOO        321632
IOWA CITY       314099
Name: City, dtype: int64

In [32]:
# DES MOINES      
df.loc[df['City'] == 'DES MOINES', 'City_DES_MOINES'] = 1
df.loc[df['City'] != 'DES MOINES', 'City_DES_MOINES'] = 0

# CEDAR RAPIDS
df.loc[df['City'] == 'CEDAR RAPIDS', 'City_CEDAR_RAPIDS'] = 1
df.loc[df['City'] != 'CEDAR RAPIDS', 'City_CEDAR_RAPIDS'] = 0

# DAVENPORT
df.loc[df['City'] == 'DAVENPORT', 'City_DAVENPORT'] = 1
df.loc[df['City'] != 'DAVENPORT' , 'City_DAVENPORT'] = 0

# WATERLOO
df.loc[df['City'] == 'WATERLOO', 'City_WATERLOO'] = 1
df.loc[df['City'] != 'WATERLOO', 'City_WATERLOO'] = 0

# IOWA CITY
df.loc[df['City'] == 'IOWA CITY', 'City_IOWA_CITY'] = 1
df.loc[df['City'] != 'IOWA CITY', 'City_IOWA_CITY'] = 0

# rest
df.loc[(df['City'] == 'DES MOINES') | (df['City'] == 'CEDAR RAPIDS'), 'Rest_Cities'] = 0

df.loc[(df['City'] != 'DES MOINES') & (df['City'] != 'CEDAR RAPIDS') & (df['City'] != 'DAVENPORT'), 'Rest_Cities'] = 1

In [33]:
df = df.drop('City', axis=1)

In [34]:
df

,Store Number,Zip Code,County Number,Category,Vendor Number,Item Number,Pack,Bottle Volume,Sale (Dollars),Volume Sold (Liters),Profit Margin,City_DES_MOINES,City_CEDAR_RAPIDS,City_DAVENPORT,City_WATERLOO,City_IOWA_CITY,Rest_Cities
0,2191,52632,56,<NA>,255,297,6,0.75,162.84,4.50,54.3,0.0,0.0,0.0,0.0,0.0,1.0
1,2205,51632,73,<NA>,255,297,6,0.75,325.68,9.00,108.6,0.0,0.0,0.0,0.0,0.0,1.0
2,3549,52627,56,<NA>,130,249,20,0.15,19.20,0.30,6.4,0.0,0.0,0.0,0.0,0.0,1.0
3,2513,522,52,<NA>,65,237,3,1.75,160.02,5.25,53.37,0.0,0.0,0.0,0.0,1.0,1.0
4,3942,52342,86,<NA>,130,249,20,0.15,19.20,0.30,6.4,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10022925,3704,536,74,1081200,434,68846,12,0.75,11.54,0.75,3.85,0.0,0.0,0.0,0.0,0.0,1.0
10022926,3704,536,74,1081200,330,80457,12,1.00,10.85,3.00,10.86,0.0,0.0,0.0,0.0,0.0,1.0
10022927,3704,536,74,1081200,260,68306,12,0.75,9.75,0.75,3.25,0.0,0.0,0.0,0.0,0.0,1.0
10022928,3704,536,74,1081100,65,67557,12,1.00,12.59,6.00,25.2,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
# check for missing values
df.isnull().sum()

Store Number                 0
Zip Code                     0
County Number                0
Category                  2047
Vendor Number                2
Item Number                  0
Pack                         0
Bottle Volume                0
Sale (Dollars)              10
Volume Sold (Liters)         0
Profit Margin                0
City_DES_MOINES              0
City_CEDAR_RAPIDS            0
City_DAVENPORT               0
City_WATERLOO                0
City_IOWA_CITY               0
Rest_Cities             448940
dtype: int64

In [36]:
df.to_csv("data2.csv")

In [37]:
df.corr()

,Store Number,Zip Code,County Number,Category,Vendor Number,Item Number,Pack,Bottle Volume,Sale (Dollars),Volume Sold (Liters),Profit Margin,City_DES_MOINES,City_CEDAR_RAPIDS,City_DAVENPORT,City_WATERLOO,City_IOWA_CITY,Rest_Cities
Store Number,1.000000,0.081268,0.005757,-0.015121,-0.005951,-0.022834,0.092503,-0.046682,-0.021175,-0.024215,-0.022208,-0.008769,-0.060559,-0.023717,0.000185,-0.063222,0.052111
Zip Code,0.081268,1.000000,-0.083010,0.006825,-0.000260,-0.000851,-0.024522,0.017567,-0.014368,-0.013421,-0.014422,-0.156035,-0.079867,-0.105142,-0.093037,0.010428,0.189114
County Number,0.005757,-0.083010,1.000000,-0.004870,0.001163,0.010187,0.017526,-0.019622,0.012156,0.004615,0.011690,0.226777,-0.002252,0.197115,-0.336103,-0.034569,-0.199691
Category,-0.015121,0.006825,-0.004870,1.000000,0.086995,0.150119,-0.042921,-0.015123,0.010642,-0.003445,0.009508,-0.007404,-0.003085,-0.002668,0.005116,-0.007633,0.008377
Vendor Number,-0.005951,-0.000260,0.001163,0.086995,1.000000,0.133233,-0.053736,0.009685,-0.009532,-0.003754,-0.010221,0.007306,0.002528,-0.000271,-0.001120,0.002634,-0.007645
Item Number,-0.022834,-0.000851,0.010187,0.150119,0.133233,1.000000,-0.012020,-0.026214,0.009394,-0.007633,0.008561,0.036812,0.001841,0.000632,0.003651,0.006395,-0.030845
Pack,0.092503,-0.024522,0.017526,-0.042921,-0.053736,-0.012020,1.000000,-0.448076,-0.044885,-0.062864,-0.046293,0.040039,0.032157,0.043229,0.045003,0.000367,-0.060111
Bottle Volume,-0.046682,0.017567,-0.019622,-0.015123,0.009685,-0.026214,-0.448076,1.000000,0.061036,0.115022,0.061188,-0.038058,-0.025730,-0.027834,-0.027913,-0.012000,0.051144
Sale (Dollars),-0.021175,-0.014368,0.012156,0.010642,-0.009532,0.009394,-0.044885,0.061036,1.000000,0.843806,0.997516,0.035758,0.004419,0.007396,0.008881,0.008712,-0.032674
Volume Sold (Liters),-0.024215,-0.013421,0.004615,-0.003445,-0.003754,-0.007633,-0.062864,0.115022,0.843806,1.000000,0.849991,0.022559,0.003891,0.004972,0.005543,0.005731,-0.021420


Just 3 columns have a great correlation with the target column. The rest of the correlations are very small. I think that at feature selection, the algorithm will select these 3 columns.

---

After cleaning the data, I can start with the Feature Engineering.
